In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data/'
base_dir = '/content/drive/MyDrive/data/'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.EfficientNetB3(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

In [ ]:
conv_base.summary()

Model: "efficientnetb3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                     

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 7, 7, 1536)       10783535  
                                                                 
 flatten_1 (Flatten)         (None, 75264)             0         
                                                                 
 dense_2 (Dense)             (None, 1024)              77071360  
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 87,863,095
Trainable params: 87,775,792
Non-trainable params: 87,303
_________________________________________________________________


In [ ]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 10

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

label_map

Found 23219 images belonging to 8 classes.
Found 2585 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
train_stop = 3604//batch
test_stop = 401//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=[tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),
                       tfa.metrics.F1Score(num_classes=8),
                       tfa.metrics.FBetaScore(num_classes=8)])

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=10)

Epoch 1/10
360/360 [==============================] - 1238s 3s/step - loss: 1.8273 - categorical_accuracy: 0.5942 - recall: 0.5758 - precision: 0.6171 - f1_score: 0.5924 - fbeta_score: 0.5924
Epoch 2/10
360/360 [==============================] - 1056s 3s/step - loss: 1.2695 - categorical_accuracy: 0.6997 - recall: 0.6844 - precision: 0.7222 - f1_score: 0.6976 - fbeta_score: 0.6976
Epoch 3/10
360/360 [==============================] - 988s 3s/step - loss: 1.2344 - categorical_accuracy: 0.7375 - recall: 0.7233 - precision: 0.7570 - f1_score: 0.7391 - fbeta_score: 0.7391
Epoch 4/10
360/360 [==============================] - 941s 3s/step - loss: 1.2161 - categorical_accuracy: 0.7436 - recall: 0.7303 - precision: 0.7605 - f1_score: 0.7438 - fbeta_score: 0.7438
Epoch 5/10
360/360 [==============================] - 920s 3s/step - loss: 1.1396 - categorical_accuracy: 0.7683 - recall: 0.7544 - precision: 0.7829 - f1_score: 0.7673 - fbeta_score: 0.7673
Epoch 6/10
360/360 [=======================

In [ ]:
test = model.evaluate(test_generator, steps=test_stop)

40/40 [==============================] - 127s 3s/step - loss: 0.4220 - categorical_accuracy: 0.9000 - recall: 0.8925 - precision: 0.9107 - f1_score: 0.9031 - fbeta_score: 0.9031


In [ ]:
model.save('/content/drive/MyDrive/dataset/ENetB3.h5')

In [ ]:
import pandas as pd

In [ ]:
df_marks = pd.DataFrame({'x_test': test_generator.filepaths,
     'y_test': test_generator.labels})

x_test = df_marks.drop(['y_test'],axis=1)
y_test = df_marks['y_test']



In [ ]:
pred = model.predict(test_generator)

259/259 [==============================] - 717s 3s/step


In [ ]:
predicted_classes = np.argmax(pred, axis = 1)
print(predicted_classes)

[0 2 6 ... 3 1 4]


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.11      0.12      0.12       322
           1       0.13      0.13      0.13       325
           2       0.14      0.11      0.12       321
           3       0.13      0.16      0.14       326
           4       0.15      0.13      0.14       322
           5       0.12      0.12      0.12       321
           6       0.12      0.13      0.13       325
           7       0.09      0.10      0.09       323

    accuracy                           0.12      2585
   macro avg       0.13      0.12      0.12      2585
weighted avg       0.13      0.12      0.12      2585

